In [1]:
import pickle as pkl
# import pandas as pd
# import yfinance as yf
# import time

# sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
# table = pd.read_html(sp500_url)
# sp500_df = table[0]  # First table is the constituents list
# tickers = sp500_df["Symbol"].tolist()

# # Yahoo uses BRK-B instead of BRK.B, same for BF-B -> BF.B
# tickers = [t.replace(".", "-") for t in tickers]


# def fetch_history(tickers, start="2020-01-01", end=None, interval="1d"):
#     all_data = []
#     chunk_size = 50  # <= 100 is safer

#     for i in range(0, len(tickers), chunk_size):
#         batch = tickers[i : i + chunk_size]
#         print(f"Fetching {batch[0]}...{batch[-1]}")

#         df = yf.download(
#             tickers=batch,
#             start=start,
#             end=end,
#             interval=interval,
#             group_by="ticker",
#             auto_adjust=False,
#             threads=True,
#         )
#         all_data.append(df)

#         time.sleep(1.5)  # avoid being throttled

#     return all_data


# history_batches = fetch_history(tickers, start="2020-01-01")
# pkl.dump(history_batches, open("./sp500.pkl", "wb"))
# # history_batches = pkl.load(open("./sp500.pkl", "rb"))

# history = pd.concat(history_batches, axis=1)

# close_df = history.xs("Close", axis=1, level=1)

# open_df = history.xs("Open", axis=1, level=1)
# high_df = history.xs("High", axis=1, level=1)
# low_df = history.xs("Low", axis=1, level=1)
# vol_df = history.xs("Volume", axis=1, level=1)

# pkl.dump(close_df, open("./sp500_close.pkl", "wb"))

In [2]:
close_df = pkl.load(open("./sp500_close.pkl", "rb"))
close_df = close_df.dropna(axis=1, how="any")
dates = close_df.index.values

In [3]:
# Convert to polars for your pipeline
import polars as pl
from backtest_lib.market.polars_impl import Axis

close_pl = pl.from_pandas(close_df)
securities = close_pl.columns
axis = Axis.from_names(securities)

In [4]:
pastview = close_pl.transpose()

pastview = pastview.rename(
    {orig: orig.replace("column_", "period_") for orig in pastview.columns}
)

window_size = 4
stagger = 0

pastview.select(pastview.columns[stagger : window_size + stagger])

period_0,period_1,period_2,period_3
f64,f64,f64,f64
94.900497,93.748497,95.143997,95.343002
68.433998,68.075996,69.890503,69.755501
187.830002,184.949997,187.119995,187.5
43.400002,43.349998,43.52,43.16
47.66,47.310001,47.580002,47.630001
…,…,…,…
30.64889,30.688889,30.644444,30.084444
23.49,23.59,23.9,23.9
259.140015,256.049988,258.01001,256.470001


In [5]:
# now with the lib
from backtest_lib.market.polars_impl import PolarsPastView
from backtest_lib.market import PastView

close_prices_df = close_pl.with_columns(pl.Series("date", dates))
past_cost_prices = PolarsPastView.from_data_frame(close_prices_df)
print(isinstance(past_cost_prices, PastView))
print(past_cost_prices.by_period[-1]["AAPL"])

True
224.89999389648438


In [6]:
print(past_cost_prices.by_security["AAPL"])

PolarsTimeseries(_vec=array([ 75.08750153,  74.35749817,  74.94999695, ..., 230.55999756,
       226.00999451, 224.8999939 ], shape=(1417,)), _axis=PeriodAxis(dt64=array(['2020-01-02T00:00:00.000000000', '2020-01-03T00:00:00.000000000',
       '2020-01-06T00:00:00.000000000', ...,
       '2025-08-19T00:00:00.000000000', '2025-08-20T00:00:00.000000000',
       '2025-08-21T00:00:00.000000000'],
      shape=(1417,), dtype='datetime64[ns]'), labels=('2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '202

In [7]:
from backtest_lib.market import BySecurity, ByPeriod
from numpy import datetime64
import datetime as dt

dt1 = dt.datetime(2025, 1, 1, 1, 1, 1)
dt2 = dt.datetime(2025, 1, 1, 1, 1, 1)

dt641 = datetime64(dt1)
dt642 = datetime64(dt2)

print(type(dt641 > dt642))


print(
    isinstance(past_cost_prices.by_security, BySecurity),
    isinstance(past_cost_prices.by_period, ByPeriod),
)

<class 'numpy.bool'>
True True


In [8]:
past_cost_prices.by_security["AAPL"].as_series()

AAPL
f64
75.087502
74.357498
74.949997
74.597504
75.797501
…
231.589996
230.889999
230.559998


In [9]:
import random
from time import perf_counter
from backtest_lib.market.polars_impl import SeriesUniverseMapping
import backtest_lib.strategy as st

import numpy as np

secs = tuple(past_cost_prices.by_period[-1].keys())
qtys = [random.randint(0, 10) for sec in secs]

holdings = SeriesUniverseMapping.from_vectors(secs, qtys)

pf = st.QuantityPortfolio(
    cash=0, holdings=holdings
)

prices = past_cost_prices.by_period[-1]

wpf = pf.into_weighted(prices)
t0 = perf_counter()
for _ in range(10000):
    wpf = pf.into_weighted(prices)
print(f"{perf_counter() - t0}")

1.2275827000012214


In [10]:
from backtest_lib.strategy import (
    MarketView,
    StrategyContext,
    WeightedPortfolio,
    Decision,
)
from backtest_lib.universe import Universe, PastUniversePrices
from backtest_lib.backtest import Backtest
from backtest_lib.market.polars_impl import SeriesUniverseMapping

market_view = MarketView(
    prices=PastUniversePrices(close=past_cost_prices), periods=dates
)
securities = tuple(securities)
universe = securities
initial_portfolio = st.WeightedPortfolio(
    cash=0,
    holdings=SeriesUniverseMapping.from_names_and_data(
        universe, pl.Series(np.tile(1 / len(securities), len(securities)))
    ),
)


def strategy(
    universe: Universe,
    market: MarketView,
    current_portfolio: WeightedPortfolio,
    ctx: StrategyContext,
):
    return Decision(initial_portfolio)


bt = Backtest(
    strategy=strategy,
    universe=universe,
    market_view=market_view,
    initial_portfolio=initial_portfolio,
)


results = bt.run()

print(f"total return: {(results.total_return - 1) * 100:.2f}%")

total return: 98.88%
